In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS,CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from multiprocessing import Pool

## import data

In [2]:
fifa = 'fifa_df_en_10k_del_url_w_stpw.csv'
g_of_t = 'game_df_en_10k_del_url_w_stpw.csv'
election = 'election_df_en_10k_del_url_w_stpw_use.csv'


fifa_machine = 'fifa_df_machine_60_perc_10k_0404.csv'
g_of_t_machine = 'game_paraphrase_df.csv'
election_machine = 'election_df_machine_60_perc_10k_0404.csv'


In [3]:
rows_per_datasets = 10000

In [4]:
fifa_df = pd.read_csv(fifa)
fifa_df_machine = pd.read_csv(fifa_machine)
fifa_df = pd.concat([fifa_df.iloc[:rows_per_datasets], fifa_df_machine.iloc[:rows_per_datasets]], axis=0, ignore_index=True)
fifa_df['clean_tweet'] = fifa_df['clean_tweet'].str.replace('worldcup2022', '')
fifa_df = fifa_df.dropna()
fifa_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,gw1 prediction qat 1 2 ecu sen 0 2 netherland...,Fifa,1,0,0,0,279,279,#WorldCup2022 GW1 Predictions\n \n#Qat 1-2 #Ec...,"['worldcup2022', 'gw1', 'prediction', 'qat', '...",67
1,this world cup be corrupt even more now so it ...,Fifa,1,1,0,0,279,300,This World Cup is corrupt even more now so it’...,"['this', 'world', 'cup', 'be', 'corrupt', 'eve...",57
2,hypocrisy it be sad that all this talk here in...,Fifa,1,0,0,0,271,271,Hypocrisy it is sad that all this talk here in...,"['hypocrisy', 'it', 'be', 'sad', 'that', 'all'...",57
3,it only come every 4 year so even though there...,Fifa,1,0,1,0,273,279,It only comes every 4 years so even though the...,"['it', 'only', 'come', 'every', '4', 'year', '...",57
4,the part about not sell alcohol at to match g...,Fifa,1,0,0,0,268,268,The part about not selling alcohol at #WorldCu...,"['the', 'part', 'about', 'not', 'sell', 'alcoh...",56
...,...,...,...,...,...,...,...,...,...,...,...
19970,cheer,Fifa,0,0,0,0,6,6,Cheers,['cheer'],1
19971,goat,Fifa,0,0,2,1,4,21,GOAT ⚽️ @Cristiano 🇵🇹,['goat'],1
19972,yesssss,Fifa,0,0,0,0,7,7,YESSSSS,['yesssss'],1
19973,germany,Fifa,0,0,0,1,7,17,@KuWallet Germany,['germany'],1


In [5]:
g_of_t_df = pd.read_csv(g_of_t)
g_of_t_df_machine = pd.read_csv(g_of_t_machine)
g_of_t_df_machine = g_of_t_df_machine.iloc[:, :11]
g_of_t_df = pd.concat([g_of_t_df.iloc[:rows_per_datasets], g_of_t_df_machine.iloc[:rows_per_datasets]], axis=0, ignore_index=True)
g_of_t_df = g_of_t_df.dropna()
g_of_t_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,look i just need to say this right now i do no...,Game,1,0,0,0,280,280,Look I just need to say this right now\nI dont...,"['look', 'i', 'just', 'need', 'to', 'say', 'th...",67
1,i m just go to say it since i feel the need to...,Game,1,0,0,0,267,267,I'm just gonna say it since I feel the need to...,"['i', 'm', 'just', 'go', 'to', 'say', 'it', 's...",65
2,just give nick a whole lecture on game of thro...,Game,1,0,0,0,268,268,Just gave nick a whole lecture on Game of Thro...,"['just', 'give', 'nick', 'a', 'whole', 'lectur...",65
3,z what do you think be next c i want to wake u...,Game,1,0,0,0,276,276,Z- what do you think is next\nC- I want to wak...,"['z', 'what', 'do', 'you', 'think', 'be', 'nex...",63
4,can you all tell i why you all hate season 8 g...,Game,1,0,0,0,269,269,Can yall tell me why yall hated season 8 game ...,"['can', 'you', 'all', 'tell', 'i', 'why', 'you...",63
...,...,...,...,...,...,...,...,...,...,...,...
19995,cersei need notch like seriously wait day voic...,Game,0,0,0,0,200,200,Cersei needs to be taken down a notch like ser...,"['cersei', 'need', 'notch', 'like', 'seriously...",41
19996,feel way tackle game throne book start slow ch...,Game,0,0,0,1,181,192,@kmcmac74 I felt the same way until I tackled ...,"['feel', 'way', 'tackle', 'game', 'throne', 'b...",41
19997,yo check petition game throne season 8 redo th...,Game,0,0,0,1,195,199,Yo @HBO just checked out this petition for a G...,"['yo', 'check', 'petition', 'game', 'throne', ...",41
19998,remember barne noble shut 2010 meet grrm book ...,Game,0,0,0,0,214,214,Remember when Barnes and Noble shut down in 20...,"['remember', 'barne', 'noble', 'shut', '2010',...",41


In [6]:
election_df = pd.read_csv(election)
election_df_machine = pd.read_csv(election_machine)
election_df = pd.concat([election_df.iloc[:rows_per_datasets], election_df_machine.iloc[:rows_per_datasets]], axis=0, ignore_index=True)
election_df['clean_tweet'] = election_df['clean_tweet'].str.replace('trump', '')
election_df['clean_tweet'] = election_df['clean_tweet'].str.replace('biden', '')
election_df['clean_tweet'] = election_df['clean_tweet'].str.replace('joebiden', '')
election_df['clean_tweet'] = election_df['clean_tweet'].str.replace('url', '')
election_df = election_df.dropna()
election_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,i m proud to be a hispanic american i study an...,Election,1.0,0,0,2,275,300,@Alon2T @realDonaldTrump Im proud to be a hisp...,"['i', 'm', 'proud', 'to', 'be', 'a', 'hispanic...",61
1,do i want to win pa yes do i believe he ll wi...,Election,1.0,0,0,0,265,265,Do I want #Biden to win PA yes Do I believe he...,"['do', 'i', 'want', 'biden', 'to', 'win', 'pa'...",61
2,i m sincerely an utterly embarrassed to be an ...,Election,1.0,1,0,0,280,298,I'm sincerely an utterly embarrassed to be an ...,"['i', 'm', 'sincerely', 'an', 'utterly', 'emba...",61
3,i m proud to be a hispanic american i study an...,Election,1.0,0,0,2,276,301,@charliekirk11 @cracosal I'm proud to be a his...,"['i', 'm', 'proud', 'to', 'be', 'a', 'hispanic...",61
4,come amp listen to a story bout a man name don...,Election,1.0,1,0,0,283,301,Come &amp; listen to a story 'bout a man named...,"['come', 'amp', 'listen', 'to', 'a', 'story', ...",60
...,...,...,...,...,...,...,...,...,...,...,...
19891,,Election,0.0,0,0,1,6,22,@leandroruschel #Trump,['trump'],1
19892,yendo,Election,0.0,0,0,1,5,17,@zaynmalik yendo,['yendo'],1
19893,,Election,0.0,0,0,0,155,155,من تلویزیون ندارم ولی خانواده من کسی تلویزیون ...,['trump'],1
19894,,Election,0.0,1,0,0,138,159,На овој мапи видимо да републиканци готово у п...,['trump'],1


In [7]:
# Merge all the dataframes
all_df = pd.concat([fifa_df, g_of_t_df, election_df], ignore_index=True)
all_df = all_df.sample(frac=1, random_state=41)
all_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
45022,view this view link joe bobulinski creepy kiss...,Election,1.0,1,0,0,144,162,View this #view #link #JoeBiden #Bobulinski #c...,"['view', 'this', 'view', 'link', 'joebiden', '...",20
38281,catch late episode game throne shake believe h...,Game,0.0,0,0,0,141,141,Just caught up on the latest episode of Game o...,"['catch', 'late', 'episode', 'game', 'throne',...",26
6446,offside qatar live up to the corruption allega...,Fifa,1.0,0,1,0,90,90,Offside… Qatar living up to the corruption all...,"['offside', 'qatar', 'live', 'up', 'to', 'the'...",11
21115,to all those people complain about game of thr...,Game,1.0,0,0,0,261,261,To all those people complaining about game of ...,"['to', 'all', 'those', 'people', 'complain', '...",52
53699,can t believe people would vandalize a jewish ...,Election,0.0,0,1,0,117,122,#Can't believe people would vandalize a Jewish...,"['can', 't', 'believe', 'people', 'would', 'va...",21
...,...,...,...,...,...,...,...,...,...,...,...
41281,do not let up do not be discourage abt long li...,Election,1.0,0,0,1,250,267,@ChristianWalk1r DO NOT LET UP Do not be disco...,"['do', 'not', 'let', 'up', 'do', 'not', 'be', ...",41
54352,make move in the forex market like a boss chec...,Election,0.0,0,1,0,109,109,Making moves in the forex market like a boss 💪...,"['make', 'move', 'in', 'the', 'forex', 'market...",19
20450,this random guy join my overwatch group and we...,Game,1.0,0,1,0,243,245,This random guy joins my overwatch group and w...,"['this', 'random', 'guy', 'join', 'my', 'overw...",55
931,a disappointing article that ignore the spurio...,Fifa,1.0,0,0,1,258,275,A disappointing article that ignores the spuri...,"['a', 'disappointing', 'article', 'that', 'ign...",37


In [8]:
def convert_to_csv_string(string_list):
    list_representation = literal_eval(string_list)
    return ' '.join(map(str, list_representation))

In [9]:
# Train svm model using clean tweets, evaluate the performance and get feature importances
def SVM_clean_tweets(features, target):
    X = features
    y = target

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Initialize SVM model
    svm_model = svm.SVC(kernel = 'sigmoid', C = 1.0, class_weight = 'balanced')

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # TF-IDF Vectorization
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Train SVM model
        svm_model.fit(X_train_tfidf, y_train)

        # Make predictions
        y_pred = svm_model.predict(X_test_tfidf)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)
    return svm_model, avg_accuracy

In [10]:
#Helper function to use multithreading in svm for modeling acceleration
def train_and_predict_svm(train_index, test_index, X_text, X_numerical, y, svm_model):
    X_train_text, X_test_text = X_text[train_index], X_text[test_index]
    X_train_numerical, X_test_numerical = X_numerical.iloc[train_index], X_numerical.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # TF-IDF Vectorization for text data
    X_train_text_tfidf = vectorizer.fit_transform(X_train_text)
    X_test_text_tfidf = vectorizer.transform(X_test_text)

    # Combine text and numerical features
    X_train_combined = hstack([X_train_text_tfidf, X_train_numerical.values])
    X_test_combined = hstack([X_test_text_tfidf, X_test_numerical.values])

    # Train the SVM model
    svm_model.fit(X_train_combined, y_train)

    # Make predictions
    y_pred = svm_model.predict(X_test_combined)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_test, y_pred

In [11]:
def SVM_tweet_emoji_profile(features, target):
    X_text = features['clean_tweet']
    X_numerical = features[['count_emoji', 'count_profile']]
    y = target

    # Initialize SVM model
    svm_model = svm.SVC(kernel='sigmoid', C=1.0)

    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    accuracies = []

    # Create a pool of worker processes
    num_cores = 32
    pool = Pool(processes=num_cores)

    # Iterate over each fold
    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_text), 1):
        # Perform parallel training and inference
        results = pool.starmap(train_and_predict_svm, [(train_index, test_index, X_text, X_numerical, y, svm_model)])

        # Collect results
        for accuracy, y_test, y_pred in results:
            accuracies.append(accuracy)
            # Print classification report for the fold
            print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Close the pool of worker processes
    pool.close()
    pool.join()

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)
    return svm_model, avg_accuracy


In [12]:
def SVM_emoji_profile_count(features, target):
    numerical_columns = ['count_emoji', 'count_profile']
    X = features[numerical_columns]
    y = target

    # Initialize SVM model
    svm_model = svm.SVC(kernel = 'sigmoid', C = 1.0)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the SVM model
        svm_model.fit(X_train, y_train)

        # Make predictions
        y_pred = svm_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)
    return svm_model, avg_accuracy

In [13]:
# Train random forest model using clean tweets, evaluate the performance and get feature importances
def RF_clean_tweets(features, target):
    X = features
    y = target

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Initialize Random Forest model
    rf_model = RandomForestClassifier(n_estimators = 100, random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # TF-IDF Vectorization
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Train rf model
        rf_model.fit(X_train_tfidf, y_train)

        # Make predictions
        y_pred = rf_model.predict(X_test_tfidf)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

    # Get feature names from the vectorizer
    feature_names = vectorizer.get_feature_names_out()
    # Get feature importances from the trained model
    feature_importances_rf = rf_model.feature_importances_
    # Pair feature names with their importances and sort
    feature_importances_rf = sorted(zip(feature_names, feature_importances_rf), key=lambda x: x[1], reverse=True)
    print('Feature importance:')
    # Print top N features and their importances
    top_n = 10
    for feature, importance in feature_importances_rf[:top_n]:
        print(f"{feature}: {importance}")

In [14]:
def RF_tweet_emoji_profile(features, target):
    # Extract text data from clean tweets
    text_columns = ['clean_tweet']
    X_text_tfidf_list = []
    y = target

    for column in text_columns:
        vectorizer = TfidfVectorizer()
        X_text_tfidf = vectorizer.fit_transform(features[column])
        X_text_tfidf_list.append(X_text_tfidf)

    # Extract emoji and profile count from the DataFrame
    numerical_columns = ['count_emoji', 'count_profile']
    X_numerical = features[numerical_columns]

    # Combine text features with numerical features
    X_combined = hstack([*X_text_tfidf_list, X_numerical])

    # Convert to dense numpy array
    X_combined_array = X_combined.toarray()

    # Initialize Random Forest model
    rf_model = RandomForestClassifier(n_estimators = 100, random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

    accuracies = []

    # Iterate over each fold
    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_combined_array), 1):
        X_train, X_test = X_combined_array[train_index], X_combined_array[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the rf model
        rf_model.fit(X_train, y_train)

        # Make predictions
        y_pred = rf_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

In [15]:
def RF_emoji_profile_count(features, target):
    numerical_columns = ['count_emoji', 'count_profile']
    X = features[numerical_columns]
    y = target

    # Initialize Random Forest model
    rf_model = RandomForestClassifier(n_estimators = 100, random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the rf model
        rf_model.fit(X_train, y_train)

        # Make predictions
        y_pred = rf_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

In [16]:
def train_and_predict_lr(train_index, test_index, X_train, y_train, X_test, y_test):
    # Initialize Logistic Regression model
    lr_model = LogisticRegression(max_iter=1000, random_state=5)

    # Train Logistic Regression model
    lr_model.fit(X_train, y_train)

    # Make predictions
    y_pred = lr_model.predict(X_test)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    coefficients = lr_model.coef_[0]
    return accuracy, y_test, y_pred, coefficients, lr_model

In [17]:
def LR_clean_tweets(features, target):
    X = features
    y = target

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=5)

    accuracies = []
    trained_models = []

    # Create a pool of worker processes
    num_cores = 32
    pool = Pool(processes=num_cores)

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train_tfidf, X_test_tfidf = vectorizer.fit_transform(X[train_index]), vectorizer.transform(X[test_index])
        y_train, y_test = y[train_index], y[test_index]

        # Perform parallel training and prediction
        results = pool.starmap(train_and_predict_lr, [(train_index, test_index, X_train_tfidf, y_train, X_test_tfidf, y_test)])

        # Collect results
        for accuracy, y_test, y_pred, coefficients, lr_model in results:
            accuracies.append(accuracy)
            trained_models.append(lr_model)

            # Print classification report for the fold
            print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

            
    # Close the pool of worker processes
    pool.close()
    pool.join()

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)
    # Get feature names from the vectorizer
    feature_names = vectorizer.get_feature_names()
    # Pair feature names with their coefficients and sort by absolute value
    feature_importances_logreg = sorted(zip(feature_names, coefficients), key=lambda x: abs(x[1]), reverse=True)
    print('Feature importance:')
    # Print top N features and their coefficients
    top_n = 10
    for feature, coef in feature_importances_logreg[:top_n]:
        print(f"{feature}: {coef}")



In [18]:
def LR_tweet_emoji_profile(features, target):
    # Extract text data from clean tweets
    text_columns = ['clean_tweet']
    X_text_tfidf_list = []
    y = target

    for column in text_columns:
        vectorizer = TfidfVectorizer()
        X_text_tfidf = vectorizer.fit_transform(features[column])
        X_text_tfidf_list.append(X_text_tfidf)

    # Extract emoji and profile count from the DataFrame
    numerical_columns = ['count_emoji', 'count_profile']
    X_numerical = features[numerical_columns]

    # Combine text features with numerical features
    X_combined = hstack([*X_text_tfidf_list, X_numerical])

    # Convert to dense numpy array
    X_combined_array = X_combined.toarray()

    # Initialize Logistic Regression model
    lr_model = LogisticRegression(max_iter = 1000, random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

    accuracies = []

    # Iterate over each fold
    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_combined_array), 1):
        X_train, X_test = X_combined_array[train_index], X_combined_array[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the lr model
        lr_model.fit(X_train, y_train)

        # Make predictions
        y_pred = lr_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

In [19]:
def LR_emoji_profile_count(features, target):
    numerical_columns = ['count_emoji', 'count_profile']
    X = features[numerical_columns]
    y = target

    # Initialize Logistic Regression model
    lr_model = LogisticRegression(max_iter = 1000, random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the lr model
        lr_model.fit(X_train, y_train)

        # Make predictions
        y_pred = lr_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred, zero_division = 1))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

In [20]:
# Train decision tree model using clean tweets, evaluate the performance and get feature importances
def DT_clean_tweets(features, target):
    X = features
    y = target

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Initialize Decision Tree model
    dt_model = DecisionTreeClassifier(random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # TF-IDF Vectorization
        X_train_tfidf = vectorizer.fit_transform(X_train)
        X_test_tfidf = vectorizer.transform(X_test)

        # Train dtr model
        dt_model.fit(X_train_tfidf, y_train)

        # Make predictions
        y_pred = dt_model.predict(X_test_tfidf)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

    # Get feature names from the TF-IDF vectorizer
    feature_names = vectorizer.get_feature_names_out()
    # Get feature importances from the trained Decision Tree model
    feature_importances_dt = dt_model.feature_importances_
    # Pair feature names with their importances
    feature_importances_dt = zip(feature_names, feature_importances_dt)
    # Sort features by their importances
    feature_importances_dt = sorted(feature_importances_dt, key=lambda x: x[1], reverse=True)
    print('Feature importance:')
    # Print top N features and their importances
    top_n = 10
    for feature, importance in feature_importances_dt[:top_n]:
        print(f"{feature}: {importance}")

In [21]:
def DT_tweet_emoji_profile(features, target):
      # Extract text data from clean tweets
    text_columns = ['clean_tweet']
    X_text_tfidf_list = []
    y = target

    for column in text_columns:
        vectorizer = TfidfVectorizer()
        X_text_tfidf = vectorizer.fit_transform(features[column])
        X_text_tfidf_list.append(X_text_tfidf)

    # Extract emoji and profile count from the DataFrame
    numerical_columns = ['count_emoji', 'count_profile']
    X_numerical = features[numerical_columns]

    # Combine text features with numerical features
    X_combined = hstack([*X_text_tfidf_list, X_numerical])

    # Convert to dense numpy array
    X_combined_array = X_combined.toarray()

    # Initialize Decision Tree model
    dt_model = DecisionTreeClassifier(random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

    accuracies = []

    # Iterate over each fold
    for fold_idx, (train_index, test_index) in enumerate(kf.split(X_combined_array), 1):
        X_train, X_test = X_combined_array[train_index], X_combined_array[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the dt model
        dt_model.fit(X_train, y_train)

        # Make predictions
        y_pred = dt_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

In [22]:
def DT_emoji_profile_count(features, target):
    numerical_columns = ['count_emoji', 'count_profile']
    X = features[numerical_columns]
    y = target

    # Initialize Decision Tree model
    dt_model = DecisionTreeClassifier(random_state = 5)

    # Initialize KFold
    kf = KFold(n_splits = 5, shuffle = True, random_state = 5)

    accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train the dt model
        dt_model.fit(X_train, y_train)

        # Make predictions
        y_pred = dt_model.predict(X_test)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

        # Print classification report for the fold
        print(f"\nClassification Report (fold {fold_idx}):\n", classification_report(y_test, y_pred))

    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    print("\nAverage Accuracy:", avg_accuracy)

## SVM model for FIFA datasets

In [22]:
SVM_clean_tweets(fifa_df['clean_tweet'], fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.77      0.73      0.75      2032
           1       0.73      0.78      0.75      1963

    accuracy                           0.75      3995
   macro avg       0.75      0.75      0.75      3995
weighted avg       0.75      0.75      0.75      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.76      0.75      0.75      1966
           1       0.76      0.77      0.77      2029

    accuracy                           0.76      3995
   macro avg       0.76      0.76      0.76      3995
weighted avg       0.76      0.76      0.76      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.76      0.72      0.74      2001
           1       0.74      0.77      0.75      1994

    accuracy                           0.75      3995
   macro avg       0.75   

(SVC(class_weight='balanced', kernel='sigmoid'), 0.7510387984981226)

In [22]:
SVM_tweet_emoji_profile(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.59      0.56      0.58      1988
           1       0.59      0.62      0.60      2007

    accuracy                           0.59      3995
   macro avg       0.59      0.59      0.59      3995
weighted avg       0.59      0.59      0.59      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.58      0.56      0.57      1972
           1       0.58      0.60      0.59      2023

    accuracy                           0.58      3995
   macro avg       0.58      0.58      0.58      3995
weighted avg       0.58      0.58      0.58      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.58      0.55      0.57      2016
           1       0.57      0.60      0.58      1979

    accuracy                           0.58      3995
   macro avg       0.58   

In [23]:
SVM_emoji_profile_count(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.64      0.41      0.50      2032
           1       0.56      0.76      0.64      1963

    accuracy                           0.58      3995
   macro avg       0.60      0.59      0.57      3995
weighted avg       0.60      0.58      0.57      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.47      0.71      0.57      1966
           1       0.45      0.24      0.31      2029

    accuracy                           0.47      3995
   macro avg       0.46      0.47      0.44      3995
weighted avg       0.46      0.47      0.44      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.48      0.71      0.57      2001
           1       0.43      0.22      0.29      1994

    accuracy                           0.47      3995
   macro avg       0.45   

## Random forest model for FIFA datasets

In [24]:
RF_clean_tweets(fifa_df['clean_tweet'], fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.72      0.77      0.74      2032
           1       0.74      0.69      0.71      1963

    accuracy                           0.73      3995
   macro avg       0.73      0.73      0.73      3995
weighted avg       0.73      0.73      0.73      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.71      0.77      0.74      1966
           1       0.76      0.70      0.73      2029

    accuracy                           0.73      3995
   macro avg       0.73      0.73      0.73      3995
weighted avg       0.73      0.73      0.73      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.70      0.77      0.73      2001
           1       0.74      0.68      0.71      1994

    accuracy                           0.72      3995
   macro avg       0.72   

In [25]:
RF_tweet_emoji_profile(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.73      0.75      0.74      1988
           1       0.74      0.73      0.74      2007

    accuracy                           0.74      3995
   macro avg       0.74      0.74      0.74      3995
weighted avg       0.74      0.74      0.74      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.73      0.78      0.75      1972
           1       0.77      0.71      0.74      2023

    accuracy                           0.75      3995
   macro avg       0.75      0.75      0.75      3995
weighted avg       0.75      0.75      0.75      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.73      0.75      0.74      2016
           1       0.74      0.72      0.73      1979

    accuracy                           0.74      3995
   macro avg       0.74   

In [26]:
RF_emoji_profile_count(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.67      0.40      0.50      2032
           1       0.56      0.79      0.66      1963

    accuracy                           0.59      3995
   macro avg       0.61      0.60      0.58      3995
weighted avg       0.61      0.59      0.58      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.61      0.42      0.50      1966
           1       0.57      0.74      0.64      2029

    accuracy                           0.58      3995
   macro avg       0.59      0.58      0.57      3995
weighted avg       0.59      0.58      0.57      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.64      0.43      0.52      2001
           1       0.57      0.75      0.65      1994

    accuracy                           0.59      3995
   macro avg       0.60   

## Logistic Regression model for FIFA datasets

In [27]:
LR_clean_tweets(fifa_df['clean_tweet'], fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.78      0.74      0.76      2032
           1       0.74      0.78      0.76      1963

    accuracy                           0.76      3995
   macro avg       0.76      0.76      0.76      3995
weighted avg       0.76      0.76      0.76      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.78      0.76      0.77      1966
           1       0.77      0.79      0.78      2029

    accuracy                           0.77      3995
   macro avg       0.77      0.77      0.77      3995
weighted avg       0.77      0.77      0.77      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.79      0.74      0.76      2001
           1       0.76      0.80      0.78      1994

    accuracy                           0.77      3995
   macro avg       0.77   

In [28]:
LR_tweet_emoji_profile(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.79      0.73      0.76      1988
           1       0.75      0.81      0.78      2007

    accuracy                           0.77      3995
   macro avg       0.77      0.77      0.77      3995
weighted avg       0.77      0.77      0.77      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.79      0.76      0.77      1972
           1       0.77      0.80      0.79      2023

    accuracy                           0.78      3995
   macro avg       0.78      0.78      0.78      3995
weighted avg       0.78      0.78      0.78      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.78      0.74      0.76      2016
           1       0.75      0.78      0.77      1979

    accuracy                           0.76      3995
   macro avg       0.76   

In [29]:
LR_emoji_profile_count(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.53      0.80      0.63      2032
           1       0.55      0.26      0.36      1963

    accuracy                           0.53      3995
   macro avg       0.54      0.53      0.50      3995
weighted avg       0.54      0.53      0.50      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.51      0.85      0.64      1966
           1       0.59      0.20      0.30      2029

    accuracy                           0.52      3995
   macro avg       0.55      0.53      0.47      3995
weighted avg       0.55      0.52      0.47      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.53      0.85      0.65      2001
           1       0.61      0.24      0.34      1994

    accuracy                           0.54      3995
   macro avg       0.57   

## Decision tree model for FIFA datasets

In [30]:
DT_clean_tweets(fifa_df['clean_tweet'], fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.64      0.67      0.65      2032
           1       0.64      0.61      0.63      1963

    accuracy                           0.64      3995
   macro avg       0.64      0.64      0.64      3995
weighted avg       0.64      0.64      0.64      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.63      0.65      0.64      1966
           1       0.65      0.62      0.64      2029

    accuracy                           0.64      3995
   macro avg       0.64      0.64      0.64      3995
weighted avg       0.64      0.64      0.64      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.65      0.68      0.67      2001
           1       0.67      0.63      0.65      1994

    accuracy                           0.66      3995
   macro avg       0.66   

In [31]:
DT_tweet_emoji_profile(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.64      0.65      0.65      1988
           1       0.65      0.64      0.65      2007

    accuracy                           0.65      3995
   macro avg       0.65      0.65      0.65      3995
weighted avg       0.65      0.65      0.65      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.64      0.67      0.65      1972
           1       0.66      0.63      0.64      2023

    accuracy                           0.65      3995
   macro avg       0.65      0.65      0.65      3995
weighted avg       0.65      0.65      0.65      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.66      0.67      0.66      2016
           1       0.66      0.64      0.65      1979

    accuracy                           0.66      3995
   macro avg       0.66   

In [32]:
DT_emoji_profile_count(fifa_df, fifa_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.67      0.40      0.50      2032
           1       0.56      0.79      0.66      1963

    accuracy                           0.59      3995
   macro avg       0.61      0.60      0.58      3995
weighted avg       0.61      0.59      0.58      3995


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.61      0.42      0.50      1966
           1       0.57      0.74      0.64      2029

    accuracy                           0.58      3995
   macro avg       0.59      0.58      0.57      3995
weighted avg       0.59      0.58      0.57      3995


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.64      0.43      0.52      2001
           1       0.57      0.75      0.65      1994

    accuracy                           0.59      3995
   macro avg       0.60   

## SVM model for Game of Thrones datasets

In [33]:
SVM_clean_tweets(g_of_t_df['clean_tweet'], g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2063
           1       1.00      1.00      1.00      1937

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1938
           1       1.00      1.00      1.00      2062

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2013
           1       1.00      1.00      1.00      1987

    accuracy                           1.00      4000
   macro avg       1.00   

In [34]:
SVM_tweet_emoji_profile(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.87      0.86      0.86      1981
           1       0.86      0.87      0.87      2019

    accuracy                           0.86      4000
   macro avg       0.86      0.86      0.86      4000
weighted avg       0.86      0.86      0.86      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.89      0.88      0.88      1993
           1       0.88      0.89      0.89      2007

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.87      0.89      0.88      2017
           1       0.88      0.87      0.87      1983

    accuracy                           0.88      4000
   macro avg       0.88   

In [35]:
SVM_emoji_profile_count(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.52      0.87      0.65      2063
           1       0.53      0.16      0.24      1937

    accuracy                           0.52      4000
   macro avg       0.52      0.51      0.45      4000
weighted avg       0.52      0.52      0.45      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.49      0.87      0.63      1938
           1       0.56      0.16      0.25      2062

    accuracy                           0.50      4000
   macro avg       0.53      0.51      0.44      4000
weighted avg       0.53      0.50      0.43      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.51      0.87      0.65      2013
           1       0.56      0.17      0.26      1987

    accuracy                           0.52      4000
   macro avg       0.54   

## Random forest model for Game of Thrones datasets

In [36]:
RF_clean_tweets(g_of_t_df['clean_tweet'], g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2063
           1       1.00      1.00      1.00      1937

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1938
           1       1.00      1.00      1.00      2062

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2013
           1       1.00      1.00      1.00      1987

    accuracy                           1.00      4000
   macro avg       1.00   

In [37]:
RF_tweet_emoji_profile(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1981
           1       1.00      1.00      1.00      2019

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1993
           1       1.00      1.00      1.00      2007

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      1.00      1.00      1983

    accuracy                           1.00      4000
   macro avg       1.00   

In [38]:
RF_emoji_profile_count(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.86      0.13      0.22      2063
           1       0.51      0.98      0.67      1937

    accuracy                           0.54      4000
   macro avg       0.69      0.55      0.45      4000
weighted avg       0.69      0.54      0.44      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.51      0.84      0.63      1938
           1       0.61      0.23      0.33      2062

    accuracy                           0.53      4000
   macro avg       0.56      0.53      0.48      4000
weighted avg       0.56      0.53      0.48      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.83      0.12      0.21      2013
           1       0.52      0.98      0.68      1987

    accuracy                           0.54      4000
   macro avg       0.68   

## Logistic Regression model for Game of Thrones datasets

In [39]:
LR_clean_tweets(g_of_t_df['clean_tweet'], g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2063
           1       1.00      1.00      1.00      1937

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1938
           1       1.00      1.00      1.00      2062

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2013
           1       1.00      0.99      1.00      1987

    accuracy                           1.00      4000
   macro avg       1.00   

In [40]:
LR_tweet_emoji_profile(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      1981
           1       1.00      0.99      1.00      2019

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      1993
           1       1.00      0.99      1.00      2007

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      0.99      1.00      1983

    accuracy                           1.00      4000
   macro avg       1.00   

In [41]:
LR_emoji_profile_count(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.87      0.13      0.22      2063
           1       0.51      0.98      0.67      1937

    accuracy                           0.54      4000
   macro avg       0.69      0.55      0.45      4000
weighted avg       0.69      0.54      0.44      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.50      0.84      0.63      1938
           1       0.60      0.22      0.32      2062

    accuracy                           0.52      4000
   macro avg       0.55      0.53      0.48      4000
weighted avg       0.55      0.52      0.47      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.53      0.85      0.65      2013
           1       0.60      0.23      0.33      1987

    accuracy                           0.54      4000
   macro avg       0.56   

## Decision Tree model for Game of Thrones datasets

In [42]:
DT_clean_tweets(g_of_t_df['clean_tweet'], g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2063
           1       1.00      1.00      1.00      1937

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1938
           1       1.00      1.00      1.00      2062

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2013
           1       1.00      1.00      1.00      1987

    accuracy                           1.00      4000
   macro avg       1.00   

In [43]:
DT_tweet_emoji_profile(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1981
           1       1.00      1.00      1.00      2019

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1993
           1       1.00      1.00      1.00      2007

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      1.00      1.00      1983

    accuracy                           1.00      4000
   macro avg       1.00   

In [44]:
DT_emoji_profile_count(g_of_t_df, g_of_t_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

           0       0.86      0.13      0.22      2063
           1       0.51      0.98      0.67      1937

    accuracy                           0.54      4000
   macro avg       0.69      0.55      0.45      4000
weighted avg       0.69      0.54      0.44      4000


Classification Report (fold 2):
               precision    recall  f1-score   support

           0       0.51      0.84      0.63      1938
           1       0.61      0.23      0.33      2062

    accuracy                           0.53      4000
   macro avg       0.56      0.53      0.48      4000
weighted avg       0.56      0.53      0.48      4000


Classification Report (fold 3):
               precision    recall  f1-score   support

           0       0.83      0.12      0.21      2013
           1       0.52      0.98      0.68      1987

    accuracy                           0.54      4000
   macro avg       0.68   

## SVM model for election dataset

In [45]:
SVM_clean_tweets(election_df['clean_tweet'], election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.77      0.65      0.71      2052
         1.0       0.68      0.80      0.74      1928

    accuracy                           0.72      3980
   macro avg       0.73      0.72      0.72      3980
weighted avg       0.73      0.72      0.72      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.73      0.62      0.67      1912
         1.0       0.69      0.79      0.74      2067

    accuracy                           0.71      3979
   macro avg       0.71      0.70      0.70      3979
weighted avg       0.71      0.71      0.71      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.74      0.66      0.70      1991
         1.0       0.69      0.77      0.73      1988

    accuracy                           0.72      3979
   macro avg       0.72   

In [22]:
SVM_tweet_emoji_profile(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.61      0.53      0.57      1958
         1.0       0.60      0.68      0.63      2022

    accuracy                           0.60      3980
   macro avg       0.60      0.60      0.60      3980
weighted avg       0.60      0.60      0.60      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.58      0.54      0.56      1964
         1.0       0.58      0.62      0.60      2015

    accuracy                           0.58      3979
   macro avg       0.58      0.58      0.58      3979
weighted avg       0.58      0.58      0.58      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.57      0.52      0.54      2003
         1.0       0.56      0.61      0.58      1976

    accuracy                           0.56      3979
   macro avg       0.56   

(SVC(kernel='sigmoid'), 0.5824778832589689)

In [23]:
SVM_emoji_profile_count(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.50      0.63      0.56      2052
         1.0       0.45      0.32      0.38      1928

    accuracy                           0.48      3980
   macro avg       0.48      0.48      0.47      3980
weighted avg       0.48      0.48      0.47      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.47      0.65      0.55      1912
         1.0       0.50      0.32      0.39      2067

    accuracy                           0.48      3979
   macro avg       0.48      0.49      0.47      3979
weighted avg       0.48      0.48      0.46      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.48      0.64      0.54      1991
         1.0       0.45      0.30      0.36      1988

    accuracy                           0.47      3979
   macro avg       0.46   

(SVC(kernel='sigmoid'), 0.47833709891503257)

## Random forest model for election datasets

In [24]:
RF_clean_tweets(election_df['clean_tweet'], election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.68      0.67      0.67      2052
         1.0       0.65      0.67      0.66      1928

    accuracy                           0.67      3980
   macro avg       0.67      0.67      0.67      3980
weighted avg       0.67      0.67      0.67      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.63      0.66      0.65      1912
         1.0       0.67      0.64      0.65      2067

    accuracy                           0.65      3979
   macro avg       0.65      0.65      0.65      3979
weighted avg       0.65      0.65      0.65      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.66      0.69      0.67      1991
         1.0       0.67      0.64      0.66      1988

    accuracy                           0.66      3979
   macro avg       0.66   

In [25]:
RF_tweet_emoji_profile(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.66      0.68      0.67      1958
         1.0       0.68      0.66      0.67      2022

    accuracy                           0.67      3980
   macro avg       0.67      0.67      0.67      3980
weighted avg       0.67      0.67      0.67      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.67      0.67      0.67      1964
         1.0       0.68      0.67      0.68      2015

    accuracy                           0.67      3979
   macro avg       0.67      0.67      0.67      3979
weighted avg       0.67      0.67      0.67      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.68      0.67      0.68      2003
         1.0       0.67      0.68      0.68      1976

    accuracy                           0.68      3979
   macro avg       0.68   

In [26]:
RF_emoji_profile_count(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.65      0.20      0.30      2052
         1.0       0.51      0.89      0.65      1928

    accuracy                           0.53      3980
   macro avg       0.58      0.54      0.47      3980
weighted avg       0.58      0.53      0.47      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.66      0.21      0.31      1912
         1.0       0.55      0.90      0.68      2067

    accuracy                           0.57      3979
   macro avg       0.60      0.55      0.50      3979
weighted avg       0.60      0.57      0.51      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.70      0.21      0.32      1991
         1.0       0.53      0.91      0.67      1988

    accuracy                           0.56      3979
   macro avg       0.62   

## Logistic Regression model for election datasets

In [27]:
LR_clean_tweets(election_df['clean_tweet'], election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.78      0.66      0.71      2052
         1.0       0.69      0.80      0.74      1928

    accuracy                           0.73      3980
   macro avg       0.73      0.73      0.73      3980
weighted avg       0.73      0.73      0.73      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.73      0.65      0.68      1912
         1.0       0.70      0.78      0.74      2067

    accuracy                           0.71      3979
   macro avg       0.71      0.71      0.71      3979
weighted avg       0.71      0.71      0.71      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.74      0.67      0.70      1991
         1.0       0.70      0.76      0.73      1988

    accuracy                           0.72      3979
   macro avg       0.72   

In [28]:
LR_tweet_emoji_profile(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.75      0.68      0.71      1958
         1.0       0.71      0.78      0.74      2022

    accuracy                           0.73      3980
   macro avg       0.73      0.73      0.73      3980
weighted avg       0.73      0.73      0.73      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.75      0.67      0.71      1964
         1.0       0.71      0.79      0.75      2015

    accuracy                           0.73      3979
   macro avg       0.73      0.73      0.73      3979
weighted avg       0.73      0.73      0.73      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.75      0.65      0.69      2003
         1.0       0.69      0.78      0.73      1976

    accuracy                           0.71      3979
   macro avg       0.72   

In [29]:
LR_emoji_profile_count(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.54      0.68      0.60      2052
         1.0       0.53      0.38      0.44      1928

    accuracy                           0.54      3980
   macro avg       0.53      0.53      0.52      3980
weighted avg       0.53      0.54      0.52      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.52      0.71      0.60      1912
         1.0       0.59      0.38      0.46      2067

    accuracy                           0.54      3979
   macro avg       0.55      0.55      0.53      3979
weighted avg       0.55      0.54      0.53      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.53      0.70      0.60      1991
         1.0       0.56      0.39      0.46      1988

    accuracy                           0.54      3979
   macro avg       0.55   

## Decision Tree model for election datasets

In [30]:
DT_clean_tweets(election_df['clean_tweet'], election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.62      0.59      0.60      2052
         1.0       0.58      0.61      0.59      1928

    accuracy                           0.60      3980
   macro avg       0.60      0.60      0.60      3980
weighted avg       0.60      0.60      0.60      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.57      0.59      0.58      1912
         1.0       0.61      0.60      0.60      2067

    accuracy                           0.59      3979
   macro avg       0.59      0.59      0.59      3979
weighted avg       0.59      0.59      0.59      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.59      0.61      0.60      1991
         1.0       0.60      0.58      0.59      1988

    accuracy                           0.60      3979
   macro avg       0.60   

In [31]:
DT_tweet_emoji_profile(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.60      0.61      0.61      1958
         1.0       0.62      0.61      0.61      2022

    accuracy                           0.61      3980
   macro avg       0.61      0.61      0.61      3980
weighted avg       0.61      0.61      0.61      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.60      0.60      0.60      1964
         1.0       0.61      0.61      0.61      2015

    accuracy                           0.60      3979
   macro avg       0.60      0.60      0.60      3979
weighted avg       0.60      0.60      0.60      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.61      0.59      0.60      2003
         1.0       0.60      0.62      0.61      1976

    accuracy                           0.60      3979
   macro avg       0.61   

In [32]:
DT_emoji_profile_count(election_df, election_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.65      0.20      0.30      2052
         1.0       0.51      0.88      0.65      1928

    accuracy                           0.53      3980
   macro avg       0.58      0.54      0.47      3980
weighted avg       0.58      0.53      0.47      3980


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.65      0.21      0.31      1912
         1.0       0.55      0.90      0.68      2067

    accuracy                           0.57      3979
   macro avg       0.60      0.55      0.50      3979
weighted avg       0.60      0.57      0.51      3979


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.70      0.21      0.32      1991
         1.0       0.53      0.91      0.67      1988

    accuracy                           0.56      3979
   macro avg       0.62   

## SVM model for combined datasets

In [33]:
SVM_clean_tweets(all_df['clean_tweet'], all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.81      0.78      0.79      5988
         1.0       0.78      0.82      0.80      5987

    accuracy                           0.80     11975
   macro avg       0.80      0.80      0.80     11975
weighted avg       0.80      0.80      0.80     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.82      0.77      0.79      6112
         1.0       0.78      0.82      0.80      5862

    accuracy                           0.80     11974
   macro avg       0.80      0.80      0.80     11974
weighted avg       0.80      0.80      0.80     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.81      0.78      0.80      5937
         1.0       0.79      0.82      0.81      6037

    accuracy                           0.80     11974
   macro avg       0.80   

In [22]:
SVM_tweet_emoji_profile(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.67      0.64      0.65      5978
         1.0       0.66      0.68      0.67      5997

    accuracy                           0.66     11975
   macro avg       0.66      0.66      0.66     11975
weighted avg       0.66      0.66      0.66     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.67      0.65      0.66      5989
         1.0       0.66      0.68      0.67      5985

    accuracy                           0.66     11974
   macro avg       0.66      0.66      0.66     11974
weighted avg       0.66      0.66      0.66     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.67      0.65      0.66      6045
         1.0       0.66      0.68      0.67      5929

    accuracy                           0.67     11974
   macro avg       0.67   

In [23]:
SVM_emoji_profile_count(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.50      0.76      0.60      5988
         1.0       0.49      0.23      0.32      5987

    accuracy                           0.49     11975
   macro avg       0.49      0.49      0.46     11975
weighted avg       0.49      0.49      0.46     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.51      0.76      0.61      6112
         1.0       0.49      0.24      0.32      5862

    accuracy                           0.51     11974
   macro avg       0.50      0.50      0.47     11974
weighted avg       0.50      0.51      0.47     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.50      0.76      0.60      5937
         1.0       0.51      0.24      0.33      6037

    accuracy                           0.50     11974
   macro avg       0.50   

## Random forest model for combined datasets

In [31]:
RF_clean_tweets(all_df['clean_tweet'], all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.80      0.78      0.79      5988
         1.0       0.79      0.81      0.80      5987

    accuracy                           0.79     11975
   macro avg       0.79      0.79      0.79     11975
weighted avg       0.79      0.79      0.79     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.81      0.77      0.79      6112
         1.0       0.77      0.81      0.79      5862

    accuracy                           0.79     11974
   macro avg       0.79      0.79      0.79     11974
weighted avg       0.79      0.79      0.79     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.80      0.79      0.79      5937
         1.0       0.79      0.81      0.80      6037

    accuracy                           0.80     11974
   macro avg       0.80   

In [11]:
RF_tweet_emoji_profile(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.49      0.58      0.53      5978
         1.0       0.49      0.41      0.44      5997

    accuracy                           0.49     11975
   macro avg       0.49      0.49      0.49     11975
weighted avg       0.49      0.49      0.49     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.50      0.59      0.54      5989
         1.0       0.50      0.42      0.46      5985

    accuracy                           0.50     11974
   macro avg       0.50      0.50      0.50     11974
weighted avg       0.50      0.50      0.50     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.50      0.58      0.54      6045
         1.0       0.49      0.42      0.46      5929

    accuracy                           0.50     11974
   macro avg       0.50   

In [33]:
RF_emoji_profile_count(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.50      0.20      0.28      5988
         1.0       0.50      0.80      0.62      5987

    accuracy                           0.50     11975
   macro avg       0.50      0.50      0.45     11975
weighted avg       0.50      0.50      0.45     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.51      0.10      0.16      6112
         1.0       0.49      0.90      0.63      5862

    accuracy                           0.49     11974
   macro avg       0.50      0.50      0.40     11974
weighted avg       0.50      0.49      0.39     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.49      0.28      0.36      5937
         1.0       0.50      0.71      0.59      6037

    accuracy                           0.50     11974
   macro avg       0.50   

## Logistic Regression model for combined datasets

In [107]:
LR_clean_tweets(all_df['clean_tweet'], all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.81      0.79      0.80      5988
         1.0       0.80      0.82      0.81      5987

    accuracy                           0.81     11975
   macro avg       0.81      0.81      0.81     11975
weighted avg       0.81      0.81      0.81     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.82      0.79      0.81      6112
         1.0       0.79      0.82      0.81      5862

    accuracy                           0.81     11974
   macro avg       0.81      0.81      0.81     11974
weighted avg       0.81      0.81      0.81     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.82      0.80      0.81      5937
         1.0       0.81      0.83      0.82      6037

    accuracy                           0.81     11974
   macro avg       0.81   

In [28]:
LR_tweet_emoji_profile(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.49      0.49      0.49      5978
         1.0       0.50      0.50      0.50      5997

    accuracy                           0.49     11975
   macro avg       0.49      0.49      0.49     11975
weighted avg       0.49      0.49      0.49     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.50      0.50      0.50      5989
         1.0       0.49      0.49      0.49      5985

    accuracy                           0.49     11974
   macro avg       0.49      0.49      0.49     11974
weighted avg       0.49      0.49      0.49     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.50      0.49      0.49      6045
         1.0       0.49      0.51      0.50      5929

    accuracy                           0.50     11974
   macro avg       0.50   

In [39]:
LR_emoji_profile_count(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.44      0.02      0.03      5988
         1.0       0.50      0.98      0.66      5987

    accuracy                           0.50     11975
   macro avg       0.47      0.50      0.35     11975
weighted avg       0.47      0.50      0.35     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.00      6112
         1.0       0.49      1.00      0.66      5862

    accuracy                           0.49     11974
   macro avg       0.74      0.50      0.33     11974
weighted avg       0.75      0.49      0.32     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.49      0.20      0.28      5937
         1.0       0.50      0.80      0.62      6037

    accuracy                           0.50     11974
   macro avg       0.49   

## Decision Tree model for combined datasets

In [28]:
DT_clean_tweets(all_df['clean_tweet'], all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.74      0.74      0.74      5988
         1.0       0.74      0.74      0.74      5987

    accuracy                           0.74     11975
   macro avg       0.74      0.74      0.74     11975
weighted avg       0.74      0.74      0.74     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.75      0.75      0.75      6112
         1.0       0.74      0.74      0.74      5862

    accuracy                           0.75     11974
   macro avg       0.75      0.75      0.75     11974
weighted avg       0.75      0.75      0.75     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.75      0.76      0.75      5937
         1.0       0.76      0.75      0.75      6037

    accuracy                           0.75     11974
   macro avg       0.75   

In [ ]:
DT_tweet_emoji_profile(all_df['clean_tweet'], all_df['Human_generated'])

It takes too much time to run this function, the result is generally lower than using clean tweets only according to previous observation.

In [12]:
DT_emoji_profile_count(all_df, all_df['Human_generated'])


Classification Report (fold 1):
               precision    recall  f1-score   support

         0.0       0.50      0.20      0.28      5988
         1.0       0.50      0.80      0.62      5987

    accuracy                           0.50     11975
   macro avg       0.50      0.50      0.45     11975
weighted avg       0.50      0.50      0.45     11975


Classification Report (fold 2):
               precision    recall  f1-score   support

         0.0       0.50      0.10      0.17      6112
         1.0       0.49      0.90      0.63      5862

    accuracy                           0.49     11974
   macro avg       0.50      0.50      0.40     11974
weighted avg       0.50      0.49      0.40     11974


Classification Report (fold 3):
               precision    recall  f1-score   support

         0.0       0.49      0.28      0.36      5937
         1.0       0.50      0.72      0.59      6037

    accuracy                           0.50     11974
   macro avg       0.50   

## Model evaulation on new datasets

The clean tweet column yields the highest accuracy for the logistic regression model among all the models. Therefore, this feature is used for evaluating the model on the new dataset.

In [23]:
mental_health = 'mental_health_df_en_10k_del_url_w_stpw.csv'
airline = 'airline_df_en_10k_del_url_w_stpw.csv'

mental_health_machine = 'mental_health_paraphrase_df.csv'
airline_machine = 'airline_paraphrase_df.csv'

In [24]:
mental_health_df = pd.read_csv(mental_health)
mental_health_machine_df = pd.read_csv(mental_health_machine)
mental_health_machine_df = mental_health_machine_df.iloc[:, :11]
mental_health_df = pd.concat([mental_health_df.iloc[:rows_per_datasets], mental_health_machine_df.iloc[:rows_per_datasets]], axis=0, ignore_index=True)
mental_health_df = mental_health_df.dropna()
mental_health_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,rt pls rt and help brave madison get her tomka...,Mental_health,1,0,0,46,3298,3945,RT @wordsforwords14 Pls RT and help brave Madi...,"['rt', 'pls', 'rt', 'and', 'help', 'brave', 'm...",462
1,rt fyi that leak riverdale script be a draft f...,Mental_health,1,5,0,23,2873,3434,RT @CWRiverdaleNews FYI that leaked Riverdale ...,"['rt', 'fyi', 'that', 'leak', 'riverdale', 'sc...",399
2,rt win my prizepackcelebrate 3yrs of http tco ...,Mental_health,1,2,0,27,2741,3098,RT @MeatFreeAthlete WIN my PrizePackCelebratin...,"['rt', 'win', 'my', 'prizepackcelebrate', '3yr...",376
3,rt have u ever want to kiss someone really bad...,Mental_health,1,0,0,7,2625,2697,RT @reIatabIe Have u ever wanted to kiss someo...,"['rt', 'have', 'u', 'ever', 'want', 'to', 'kis...",369
4,rt right wing independent greek party agree to...,Mental_health,1,0,0,4,2441,2516,RT @BBCBreaking Right-wing Independent Greeks ...,"['rt', 'right', 'wing', 'independent', 'greek'...",287
...,...,...,...,...,...,...,...,...,...,...,...
19920,sofa shopping yay,Mental_Health,0,0,0,0,17,18,Sofa shopping Yay,"['sofa', 'shopping', 'yay']",3
19921,round 2 tonight,Mental_Health,0,0,0,1,15,29,@bayleeideker round 2 tonight,"['round', '2', 'tonight']",3
19922,so freak stoke,Mental_Health,0,0,0,1,18,24,@5SOS so freaking stoked,"['so', 'freak', 'stoke']",3
19923,chill at home,Mental_Health,0,0,0,1,16,29,@lexiloungin chilling at home,"['chill', 'at', 'home']",3


In [25]:
airline_df = pd.read_csv(airline)
airline_machine_df = pd.read_csv(airline_machine)
airline_machine_df = airline_machine_df.iloc[:, :11]
airline_df = pd.concat([airline_df.iloc[:rows_per_datasets], airline_machine_df.iloc[:rows_per_datasets]], axis=0, ignore_index=True)
airline_df = airline_df.dropna()
airline_df

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,ahah that be why i love sw 2015 02 20 171842 0...,Airline,1,0,0,12,1546,1701,@SouthwestAir Ahah That is why\nI love SW^^201...,"['ahah', 'that', 'be', 'why', 'i', 'love', 'sw...",208
1,can i put sun in my carry on rt united right n...,Airline,1,0,0,6,1005,1050,Can I put sun in my carry on RT united @ant_kn...,"['can', 'i', 'put', 'sun', 'in', 'my', 'carry'...",134
2,noooo don t do it please don t do it rt our fl...,Airline,1,0,0,2,735,751,Noooo don't do it please don't do it RT @JetBl...,"['noooo', 'don', 't', 'do', 'it', 'please', 'd...",98
3,by the grace of god i make it usairway we don ...,Airline,1,0,0,3,605,633,BY THE GRACE OF GOD I MADE IT USAirways @__RWG...,"['by', 'the', 'grace', 'of', 'god', 'i', 'make...",98
4,stop the madness our fleet s on fleek http tco...,Airline,1,0,0,3,612,636,Stop the madness @JetBlue Our fleet's on fleek...,"['stop', 'the', 'madness', 'our', 'fleet', 's'...",68
...,...,...,...,...,...,...,...,...,...,...,...
19958,do and do thank for the help,Airline,0,0,0,1,33,41,@united Done and done thanks for the help,"['do', 'and', 'do', 'thank', 'for', 'the', 'he...",7
19959,line up lax avgeek http tco djhxvjt201,Airline,0,0,0,3,43,74,@SouthwestAir line up @LAX #LAX @AirlineGeeks ...,"['line', 'up', 'lax', 'avgeek', 'http', 'tco',...",7
19960,i ain t ever fly again getmeouttahere,Airline,0,0,0,2,42,58,@JetBlue I ain't ever flying @United again #ge...,"['i', 'ain', 't', 'ever', 'fly', 'again', 'get...",7
19961,great chat with you appreciate the help,Airline,0,0,0,2,43,62,@united @annricord great chatting with you app...,"['great', 'chat', 'with', 'you', 'appreciate',...",7


In [31]:
def preprocess_text(text):
    return text.lower()

In [32]:
# Define a function to train Logistic Regression model
def train_lr_model(X_train, y_train):
    lr_model = LogisticRegression(max_iter = 1000, random_state = 5)
    lr_model.fit(X_train, y_train)
    return lr_model

In [33]:
# Evaluate LR model on new data
def evaluate_lr_model(lr_model, X_test, y_test, accuracies):
    y_pred = lr_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    accuracies.append(accuracy)

In [34]:
# Define a function to train LR model using KFold cross-validation
def train_lr_with_kfold(features, target, vectorizer, num_folds = 5):
    X = features
    y = target

    kf = KFold(n_splits = num_folds, shuffle = True, random_state = 5)

    accuracies = []
    trained_models = []

    # Create a pool of worker processes
    num_cores = 32
    pool = Pool(processes=num_cores)

    for fold_idx, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train_tfidf = vectorizer.transform(X.iloc[train_index])
        X_test_tfidf = vectorizer.transform(X.iloc[test_index])
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Perform parallel training and prediction
        results = pool.starmap(train_lr_model, [(X_train_tfidf, y_train)])

        for lr_model in results:
            trained_models.append(lr_model)

            # Evaluate the trained model
            accuracy = lr_model.score(X_test_tfidf, y_test)
            accuracies.append(accuracy)


    # Close the pool of worker processes
    pool.close()
    pool.join()

    return trained_models

## Mental health dataset evaluation

In [35]:
# Extract features and target from datasets
all_df_features = all_df['clean_tweet']
all_df_target = all_df['Human_generated']

mental_health_df_features = mental_health_df['clean_tweet']
mental_health_df_target = mental_health_df['Human_generated']

# Combine text from both datasets
combined_text = pd.concat([all_df_features, mental_health_df_features], axis=0)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

vectorizer.fit(combined_text.apply(preprocess_text))

trained_models = train_lr_with_kfold(all_df_features, all_df_target, vectorizer)

accuracies = []
# Evaluate the trained models on mental_health_df
for idx, lr_model in enumerate(trained_models, 1):
    print(f"\nEvaluation of Model {idx} on mental_health_df:")
    X_mental_health_df = vectorizer.transform(mental_health_df_features.apply(preprocess_text))
    evaluate_lr_model(lr_model, X_mental_health_df, mental_health_df_target, accuracies)
    
average_accuracy = sum(accuracies) / len(accuracies)
print("\nAverage Accuracy:", average_accuracy)


Evaluation of Model 1 on mental_health_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.71      0.67      9925
           1       0.67      0.58      0.62     10000

    accuracy                           0.65     19925
   macro avg       0.65      0.65      0.64     19925
weighted avg       0.65      0.65      0.64     19925


Evaluation of Model 2 on mental_health_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.71      0.67      9925
           1       0.67      0.59      0.63     10000

    accuracy                           0.65     19925
   macro avg       0.65      0.65      0.65     19925
weighted avg       0.65      0.65      0.65     19925


Evaluation of Model 3 on mental_health_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.72      0.67      9925
           1       0.67      0

## Airline dataset evaluation

In [38]:
# Extract features and target from datasets
airline_df_features = airline_df['clean_tweet']
airline_df_target = airline_df['Human_generated']

# Combine text from both datasets
combined_text = pd.concat([all_df_features, airline_df_features], axis=0)

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

vectorizer.fit(combined_text.apply(preprocess_text))

trained_models = train_lr_with_kfold(all_df_features, all_df_target, vectorizer)

accuracies = []

# Evaluate the trained models on airline_df
for idx, lr_model in enumerate(trained_models, 1):
    print(f"\nEvaluation of Model {idx} on airline_df:")
    X_airline_df = vectorizer.transform(airline_df_features.apply(preprocess_text))
    evaluate_lr_model(lr_model, X_airline_df, airline_df_target, accuracies)

average_accuracy = sum(accuracies) / len(accuracies)
print("\nAverage Accuracy:", average_accuracy)


Evaluation of Model 1 on airline_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.67      0.68      9963
           1       0.68      0.70      0.69     10000

    accuracy                           0.69     19963
   macro avg       0.69      0.69      0.69     19963
weighted avg       0.69      0.69      0.69     19963


Evaluation of Model 2 on airline_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.67      0.68      9963
           1       0.68      0.71      0.69     10000

    accuracy                           0.69     19963
   macro avg       0.69      0.69      0.69     19963
weighted avg       0.69      0.69      0.69     19963


Evaluation of Model 3 on airline_df:

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.67      0.68      9963
           1       0.68      0.71      0.69     